In [100]:
import pandas as pd
TrainingLabels = pd.read_csv('Training labels.csv', index_col='id')
TrainingFeatures = pd.read_csv('Training features.csv', index_col='id')
TestFeatures = pd.read_csv('Test features.csv', index_col='id')



In [101]:
preprocess = pd.concat([TrainingFeatures,TestFeatures])
#preprocess

In [102]:
preprocess.drop(['wpt_name','num_private','region','recorded_by',
'extraction_type_class','extraction_type','payment_type','water_quality','quantity_group','source_type',
'source','waterpoint_type','date_recorded','management_group','scheme_name'],axis=1,inplace=True)
preprocess.columns

Index(['amount_tsh', 'funder', 'gps_height', 'installer', 'longitude',
       'latitude', 'basin', 'subvillage', 'region_code', 'district_code',
       'lga', 'ward', 'population', 'public_meeting', 'scheme_management',
       'permit', 'construction_year', 'extraction_type_group', 'management',
       'payment', 'quality_group', 'quantity', 'source_class',
       'waterpoint_type_group'],
      dtype='object')

In [103]:
import numpy as nan
preprocess['funder'] = preprocess['funder'].str.lower()
preprocess['funder'].fillna(preprocess.installer, inplace=True)
preprocess['funder'].fillna('nan', inplace=True)
preprocess['funder'].replace(to_replace=('0', '-', ''), value='nan', inplace=True)
funder_reduction = ['kkkt','vision','foundat','tlc','wsd','norad','sema','wfp','ded','unicef','danida','hes','rws','dhv','isf']

for category in funder_reduction:
    subCategory = preprocess[preprocess['funder'].str.contains(category)]
    unique = subCategory['funder'].unique()
    preprocess['funder'].replace(to_replace=unique,value=category, inplace=True)

multi = [
[['hifab','amref','oxfa','wateraid','Wate Aid','Water Aid','adra','plan','Ces(gmbh)','Ces (gmbh)','shipo','dmdd','oikos'],'project'],
[['mzee','priva','mwita','ltd','bank','kanis'],'private'],
[['municipal council', 'distric','govern','ministry','supply','adb','rudep'],'govern'],
[['rc','roman',' mission','cathoric','st ','churc','anglican','tcrs'],'church'],
[['total land care','total landcare','totaland care','totoland','totoland care'],'totallandcare'],
[['tassaf','tasaf'],'tasaf'],
[['ngo ',' ngo'],'ngo'],
[['rotary','club'],'club'],
[['islam','muslim'],'muslim'],
[['said ',' said'],'said'],
[['british','japan','egypt','belgian','italy','iran','china','finland','irish','swedish','german','netherlan','european','kuwait','african','jaica','jica'],'foreign'],
[['village','community','rural','halmashauri','lga'],'village'],
[['lga','adb'],'lga & adb'],
[['Fin Water','Fini Water','Finn Water','Finw','Finwater','Fiwater'],'fini']
]
for category in multi:
    for name in category[0]:
        subCategory = preprocess.loc[preprocess['funder'].str.contains(name)]
        unique = subCategory['funder'].unique()
        preprocess['funder'].replace(to_replace=unique,value=category[1], inplace=True)

preprocess['funder'].replace(to_replace=('mission','missio','missi'),value='church', inplace=True)


count = preprocess.groupby('funder')['funder'].count()
other_list = count[count<100].index.tolist()
preprocess['funder'].replace(to_replace=other_list, value='other', inplace=True)  

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(preprocess.groupby('funder')['funder'].count().sort_values(ascending=False))

C:\Users\Yasith\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


funder
govern                     15431
other                      10791
nan                         5419
danida                      3921
project                     3661
private                     3402
foreign                     3163
church                      3077
hes                         2802
kkkt                        1953
rws                         1742
vision                      1603
village                     1430
unicef                      1350
tasaf                       1316
dhv                         1146
norad                       1078
dwsp                        1012
water                        739
dwe                          605
fini water                   474
isf                          429
wsd                          322
ces(gmbh)                    316
finw                         281
dh                           261
ded                          246
kiliwater                    243
magadini-makiwaru water      226
w.b                          223
go 

In [104]:
#pd.crosstab(index=preprocess.scheme_management,columns=preprocess.management)
preprocess['management_cross'] = preprocess['scheme_management'] + preprocess['management']
count = preprocess.groupby('management_cross')['management_cross'].count()
other_list = count[count<100].index.tolist()
preprocess['management_cross'].replace(to_replace=other_list, value='other', inplace=True)  
preprocess.drop(['scheme_management','management'],axis=1,inplace=True)
#preprocess.groupby('management_cross')['management_cross'].count()


In [105]:
import numpy as np

preprocess['gps_height'] = preprocess['gps_height'].abs()
preprocess['gps_height'].replace(to_replace=0, value=np.nan, inplace=True)
preprocess['longitude'].replace(to_replace=0, value=np.nan, inplace=True)
preprocess['construction_year'].replace(to_replace=0, value=np.nan, inplace=True)
preprocess['age'] = 2013 - preprocess.construction_year
preprocess.drop('construction_year', axis = 1, inplace = True)

# pop_bins = [-1,0,50, 200, 31000]
# pop_labels = [1,2,3,4]
# TrainingFeatures['pop_binned'] = pd.cut(TrainingFeatures['population'], bins=pop_bins, labels=pop_labels)
# preprocess.drop('population', axis = 1, inplace = True)

preprocess.drop('installer', axis = 1, inplace = True)

bool_mapper = {"False":0, "True":1}
preprocess["permit"] = preprocess["permit"].replace(bool_mapper)
preprocess["public_meeting"] = preprocess["public_meeting"].replace(bool_mapper)


In [106]:
preprocess.isna().sum()

amount_tsh                   0
funder                       0
gps_height               25649
longitude                 2269
latitude                     0
basin                        0
subvillage                 470
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            4155
permit                    3793
extraction_type_group        0
payment                      0
quality_group                0
quantity                     0
source_class                 0
waterpoint_type_group        0
management_cross          4846
age                      25969
dtype: int64

In [107]:
preprocess["permit"] = preprocess["permit"].astype('float')
preprocess["public_meeting"] = preprocess["public_meeting"].astype('float')

In [108]:
for fill in ['age','longitude','gps_height']:
    preprocess[fill].fillna(preprocess[fill].mean(), inplace=True)

for fill in ['public_meeting','permit']:
    preprocess[fill].fillna(preprocess[fill].mode()[0], inplace=True)

for fill in ['management_cross','subvillage']:
    preprocess[fill].fillna('nan',inplace=True)

In [109]:
preprocess["permit"] = preprocess["permit"].astype('uint8')
preprocess["public_meeting"] = preprocess["public_meeting"].astype('uint8')

In [110]:
object_columns = preprocess.select_dtypes(include=['object']).columns
for object_col in object_columns:
    preprocess[object_col] = preprocess[object_col].astype('category')

In [111]:
preprocess.isna().sum()

amount_tsh               0
funder                   0
gps_height               0
longitude                0
latitude                 0
basin                    0
subvillage               0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
permit                   0
extraction_type_group    0
payment                  0
quality_group            0
quantity                 0
source_class             0
waterpoint_type_group    0
management_cross         0
age                      0
dtype: int64

In [112]:
# import matplotlib.pyplot as plt

# import seaborn as sns
# from sklearn.feature_selection import mutual_info_regression


# # Set Matplotlib defaults
# plt.style.use("seaborn-whitegrid")
# plt.rc("figure", autolayout=True)
# plt.rc(
#     "axes",
#     labelweight="bold",
#     labelsize="large",
#     titleweight="bold",
#     titlesize=14,
#     titlepad=10,
# )

# def make_mi_scores(X, y):
#     #X = X.copy()
#     #for colname in X.select_dtypes(["object", "category"]):
#     #    X[colname], _ = X[colname].factorize()
#     # All discrete features should now have integer dtypes
#     discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
#     mi_scores = mutual_info_regression(X, y)
#     mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
#     mi_scores = mi_scores.sort_values(ascending=False)
#     return mi_scores


# def plot_mi_scores(scores):
#     scores = scores.sort_values(ascending=True)
#     width = np.arange(len(scores))
#     ticks = list(scores.index)
#     plt.barh(width, scores)
#     plt.yticks(width, ticks)
#     plt.title("Mutual Information Scores")

# mi_scores = make_mi_scores(preprocess.iloc[:59400,:],TrainingLabels)

# print(mi_scores)

# plt.figure(dpi=100, figsize=(8, 5))
# plot_mi_scores(mi_scores)

In [113]:
#preprocess.reset_index(drop=True, inplace=True)
TrainingFeatures = preprocess.iloc[:59400,:]
TestFeatures = preprocess.iloc[59400:,:]
#TrainingLabels.reset_index(drop=True, inplace=True)
TrainingFeatures.to_csv('training_cleaned_mode.csv', index=True)
TestFeatures.to_csv('test_cleaned_mode.csv', index=True)

In [114]:
class UciAdultClassifierObjective(object):
    def __init__(self, dataset, const_params, fold_count):
        self._dataset = dataset
        self._const_params = const_params.copy()
        self._fold_count = fold_count
        self._evaluated_count = 0
        
    def _to_catboost_params(self, hyper_params):
        return {
            'learning_rate': hyper_params['learning_rate'],
            'depth': hyper_params['depth'],
            'l2_leaf_reg': hyper_params['l2_leaf_reg']}
    
    # hyperopt optimizes an objective using `__call__` method (e.g. by doing 
    # `foo(hyper_params)`), so we provide one
    def __call__(self, hyper_params):
        # join hyper-parameters provided by hyperopt with hyper-parameters 
        # provided by the user
        params = self._to_catboost_params(hyper_params)
        params.update(self._const_params)
        
        print('evaluating params={}'.format(params), file=sys.stdout)
        sys.stdout.flush()
        
        # we use cross-validation for objective evaluation, to avoid overfitting
        scores = cb.cv(
            pool=self._dataset,
            params=params,
            fold_count=self._fold_count,
            partition_random_seed=20181224,
            verbose=False)
        
        # scores returns a dictionary with mean and std (per-fold) of metric 
        # value for each cv iteration, we choose minimal value of objective 
        # mean (though it will be better to choose minimal value among all folds)
        # because noise is additive
        max_mean_auc = np.max(scores['test-AUC-mean'])
        print('evaluated score={}'.format(max_mean_auc), file=sys.stdout)
        
        self._evaluated_count += 1
        print('evaluated {} times'.format(self._evaluated_count), file=sys.stdout)
        
        # negate because hyperopt minimizes the objective
        return {'loss': -max_mean_auc, 'status': hyperopt.STATUS_OK}

In [115]:
def find_best_hyper_params(dataset, const_params, max_evals=100):    
    # we are going to optimize these three parameters, though there are a lot more of them (see CatBoost docs)
    parameter_space = {
        'learning_rate': hyperopt.hp.uniform('learning_rate', 0.2, 1.0),
        'depth': hyperopt.hp.randint('depth', 7),
        'l2_leaf_reg': hyperopt.hp.uniform('l2_leaf_reg', 1, 10)}
    objective = UciAdultClassifierObjective(dataset=dataset, const_params=const_params, fold_count=6)
    trials = hyperopt.Trials()
    best = hyperopt.fmin(
        fn=objective,
        space=parameter_space,
        algo=hyperopt.rand.suggest,
        max_evals=max_evals,
        rstate=np.random.RandomState(seed=20181224))
    return best

def train_best_model(X, y, const_params,cat_col, max_evals=100, use_default=False):
    # convert pandas.DataFrame to catboost.Pool to avoid converting it on each 
    # iteration of hyper-parameters optimization
    dataset = cb.Pool(X, y, cat_features=cat_col)
    
    if use_default:
        # pretrained optimal parameters
        best = {
            'learning_rate': 0.4234185321620083, 
            'depth': 5, 
            'l2_leaf_reg': 9.464266235679002}
    else:
        best = find_best_hyper_params(dataset, const_params, max_evals=max_evals)
    
    # merge subset of hyper-parameters provided by hyperopt with hyper-parameters 
    # provided by the user
    hyper_params = best.copy()
    hyper_params.update(const_params)
    
    # drop `use_best_model` because we are going to use entire dataset for 
    # training of the final model
    hyper_params.pop('use_best_model', None)
    
    model = cb.CatBoostClassifier(**hyper_params)
    model.fit(dataset, verbose=False)
    
    return model, hyper_params

In [116]:
preprocess.select_dtypes(include=['category']).columns

Index(['funder', 'basin', 'subvillage', 'lga', 'ward', 'extraction_type_group',
       'payment', 'quality_group', 'quantity', 'source_class',
       'waterpoint_type_group', 'management_cross'],
      dtype='object')

In [117]:
cat_columns = ['funder', 'basin', 'subvillage', 'lga', 'ward', 'extraction_type_group',
       'payment', 'quality_group', 'quantity', 'source_class',
       'waterpoint_type_group', 'management_cross']
type(cat_columns)

list

In [118]:
import catboost as cb
# make it True if your want to use GPU for training
have_gpu = True
# skip hyper-parameter optimization and just use provided optimal parameters
use_optimal_pretrained_params = True
# number of iterations of hyper-parameter search
hyperopt_iterations = 30

const_params = dict({
    'task_type': 'GPU' if have_gpu else 'CPU',
    'iterations': 1000,
    'random_seed': 20181224})

model, params = train_best_model(
    TrainingFeatures, TrainingLabels, 
    const_params, 
    cat_columns,
    max_evals=hyperopt_iterations, 
    use_default=use_optimal_pretrained_params)
#print('best params are {}'.format(params), file=sys.stdout)

In [122]:
model.best_score_

{'learn': {'MultiClass': 0.4052188880997475}}

In [20]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from hpsklearn import HyperoptEstimator

#cat = CatBoostClassifier(cat_features=cat_columns)
#print (cross_val_score(cat, TrainingFeatures, TrainingLabels))
cat = HyperoptEstimator()
cat.fit(TrainingFeatures, TrainingLabels)


  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

job exception: "None of [Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,\n                9,\n            ...\n            47510, 47511, 47512, 47513, 47514, 47515, 47516, 47517, 47518,\n            47519],\n           dtype='int64', length=47520)] are in the [columns]"



  0%|          | 0/1 [00:01<?, ?trial/s, best loss=?]


KeyError: "None of [Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,\n                9,\n            ...\n            47510, 47511, 47512, 47513, 47514, 47515, 47516, 47517, 47518,\n            47519],\n           dtype='int64', length=47520)] are in the [columns]"

In [ ]:
# import eli5
# from eli5.sklearn import PermutationImportance

# perm = PermutationImportance(cat, random_state=1).fit(val_X, val_y)
# eli5.show_weights(perm, feature_names = val_X.columns.tolist())

cat.best_model()

Weight,Feature
0.0599 ± 0.0064,quantity_non functional
0.0445 ± 0.0057,installer_non functional
0.0216 ± 0.0026,quantity_functional needs repair
0.0209 ± 0.0025,age
0.0196 ± 0.0040,latitude
0.0183 ± 0.0030,waterpoint_type_non functional
0.0178 ± 0.0026,longitude
0.0173 ± 0.0035,extraction_type_non functional
0.0121 ± 0.0008,installer_functional needs repair
0.0111 ± 0.0025,gps_height


In [123]:
s = model.predict(TestFeatures)
#s = le.inverse_transform(s)
seas_df = pd.DataFrame(s, columns = ['status_group'], index = TestFeatures.index)

seas_df.to_csv('output_yasith_1.csv', index=True)

In [ ]:
def calculate_score_on_dataset_and_show_graph(X, y, model):
    import sklearn.metrics
    import matplotlib.pylab as pl
    pl.style.use('ggplot')
    
    dataset = cb.Pool(X, y, cat_features=np.where(X.dtypes != np.float)[0])
    fpr, tpr, _ = cbu.get_roc_curve(model, dataset, plot=True)
    auc = sklearn.metrics.auc(fpr, tpr)
    return auc
